In [1]:
import os
os.chdir("../")
%pwd

'/home/dheeraj37/Projects/Audio-Genre-Classification-1'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    model_path: Path
    trained_model_path: Path
    train_data_path: Path
    val_data_path: Path
    test_data_path: Path
    params_epochs: int
    params_batch_size: int
    params_shuffle: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path



In [3]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


2023-09-24 00:17:06.440717: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-24 00:17:06.441911: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 00:17:06.463539: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 00:17:06.464335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 00:17:06.860673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callback_config


    def get_training_config(self) -> TrainingConfig:
        model_training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model
        prepare_data = self.config.prepare_data
        params = self.params
        create_directories([
            Path(model_training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(model_training.root_dir),
            model_path=Path(prepare_base_model.model_path),
            trained_model_path=Path(model_training.trained_model_path),
            train_data_path=Path(prepare_data.train_data_path), 
            test_data_path=Path(prepare_data.test_data_path),
            val_data_path=Path(prepare_data.val_data_path),     
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_shuffle=params.SHUFFLE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config



In [5]:
import time

In [6]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from cnnClassifier.utils.common import MFCCDataGenerator

from sklearn.model_selection import train_test_split

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        
    def get_model(self):
        self.model = tf.keras.models.load_model(
            self.config.model_path
        )

    def get_datasets(self):
        self.train_df = pd.read_json(self.config.train_data_path)
        self.val_df = pd.read_json(self.config.val_data_path)
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path, save_format="h5")


    def train(self, callback_list: list):
        self.train_data_generator = MFCCDataGenerator(self.train_df, batch_size=64, shuffle=True)
        self.val_data_generator = MFCCDataGenerator(self.val_df, batch_size=64, shuffle=False)

        #self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        #self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.history = self.model.fit(
            self.train_data_generator,
            epochs=self.config.params_epochs,
            #steps_per_epoch=self.steps_per_epoch,
            #validation_steps=self.validation_steps,
            validation_data=self.val_data_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_model()
    training.get_datasets()
    #training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-09-24 00:18:45,810]: INFO: common: yaml file: config/config.yaml loaded successfully
[2023-09-24 00:18:45,812]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-24 00:18:45,812]: INFO: common: created directory at: artifacts
[2023-09-24 00:18:45,813]: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir
[2023-09-24 00:18:45,813]: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[2023-09-24 00:18:45,813]: INFO: common: created directory at: artifacts/model_training
Epoch 1/100
64/64 [==============================] - 2s 33ms/step - loss: 3.1513 - accuracy: 0.1445 - val_loss: 2.5932 - val_accuracy: 0.1197
Epoch 2/100
64/64 [==============================] - 2s 31ms/step - loss: 2.4770 - accuracy: 0.2002 - val_loss: 2.0548 - val_accuracy: 0.2650
Epoch 3/100
64/64 [==============================] - 2s 30ms/step - loss: 2.2178 - accuracy: 0.2427 - val_loss: 1.8364 - val_accuracy: 0.3768
Epoch 4/100
64/64 [==

KeyboardInterrupt: 

In [ ]:
def train_valid_generator(self): 

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )